### GROUP, UNWIND, ALIAS ...

### Configuration

In [1]:
import pymongo
from pprint import pprint
import pandas as pd
import numpy as np
from IPython.display import Image

In [2]:
connection = pymongo.MongoClient('localhost',27017)

In [3]:
db = connection.python

col = db.sales

In [4]:
#간단한 정보확인
for i in col.find().limit(1):
    pprint(i)

{'_id': {'id': '5bd761dcae323e45a93ccfe8'},
 'couponUsed': True,
 'customer': {'age': {'$numberInt': '42'},
              'email': 'cauho@witwuta.sv',
              'gender': 'M',
              'satisfaction': {'$numberInt': '4'}},
 'items': [{'name': 'printer paper',
            'price': {'$numberDecimal': '40.01'},
            'quantity': {'$numberInt': '2'},
            'tags': ['office', 'stationary']},
           {'name': 'notepad',
            'price': {'$numberDecimal': '35.29'},
            'quantity': {'$numberInt': '2'},
            'tags': ['office', 'writing', 'school']},
           {'name': 'pens',
            'price': {'$numberDecimal': '56.12'},
            'quantity': {'$numberInt': '5'},
            'tags': ['writing', 'office', 'school', 'stationary']},
           {'name': 'backpack',
            'price': {'$numberDecimal': '77.71'},
            'quantity': {'$numberInt': '2'},
            'tags': ['school', 'travel', 'kids']},
           {'name': 'notepad',
         

### Goal!

- \$match 
- \$project
- \$sort
- \$skip
- \$limit

In [5]:

Image(url= "https://appsyoda.com/blogimages/MongoDB-Pipeline.png")

#### When using find

In [6]:
for i in col.find({"couponUsed":True,"customer.gender":"M"},{"_id":0,"customer.satisfaction":1}).limit(2):
    pprint(i)

{'customer': {'satisfaction': {'$numberInt': '4'}}}
{'customer': {'satisfaction': {'$numberInt': '5'}}}


#### When using aggregate 

In [80]:
for i in col.aggregate([{"$match":{"couponUsed":True,"customer.gender":"M"}},{"$project":{"_id":0,"Sat":"$customer.satisfaction"}},{"$limit":2}]):
    pprint(i)

{'Sat': {'$numberInt': '4'}}
{'Sat': {'$numberInt': '5'}}


In [38]:
print("\$")

\$


#### This may seem pretty much the same, but...

In [79]:
for i in col.aggregate([{"$limit":2},{"$match":{"couponUsed":True,"customer.gender":"M"}},{"$project":{"_id":0,"customer.satisfaction":1}}]):
    pprint(i) 

{'customer': {'satisfaction': {'$numberInt': '4'}}}


#### As you can see, we can customize order of commands.

---


### 오프라인으로 구매하는 사람 비율 vs 온라인으로 구매하는 사람 비율을 알아보자
- 그룹화
- 카운트

In [9]:
#몇가지 방법이 있는가? 확인
col.find().distinct("purchaseMethod")

['In store', 'Online', 'Phone']

In [10]:
#프로젝트를 통해 표현할 것을 지정
#하나의 레코드가 1로 지정되어야 하므로 1씩 넣어준다 (literal사용)

for i in col.aggregate([{"$project":{"purchaseMethod":1,"count":{"$literal":1}}},
                        {"$group":{"_id":
                                   {"Method":"$purchaseMethod"},
                                   "HowMany":{"$sum":"$count"}}}]):
    print(i)

   

{'_id': {'Method': 'In store'}, 'HowMany': 2819}
{'_id': {'Method': 'Online'}, 'HowMany': 1585}
{'_id': {'Method': 'Phone'}, 'HowMany': 596}


In [11]:
#alias쓰는 법 복습 & unwind

for i in col.aggregate([{"$project":{"_id":0,"CouponUser":"$couponUsed","Category":"$items.name","Prices":"$items.price"}},{"$limit":1},{"$unwind":"$Prices"}]):
    pprint(i)


{'Category': ['printer paper',
              'notepad',
              'pens',
              'backpack',
              'notepad',
              'envelopes',
              'envelopes',
              'binder'],
 'CouponUser': True,
 'Prices': {'$numberDecimal': '40.01'}}
{'Category': ['printer paper',
              'notepad',
              'pens',
              'backpack',
              'notepad',
              'envelopes',
              'envelopes',
              'binder'],
 'CouponUser': True,
 'Prices': {'$numberDecimal': '35.29'}}
{'Category': ['printer paper',
              'notepad',
              'pens',
              'backpack',
              'notepad',
              'envelopes',
              'envelopes',
              'binder'],
 'CouponUser': True,
 'Prices': {'$numberDecimal': '56.12'}}
{'Category': ['printer paper',
              'notepad',
              'pens',
              'backpack',
              'notepad',
              'envelopes',
              'envelopes',
          

In [29]:
#alias쓰는 법 복습 & unwind

for i in col.aggregate([{"$project":{"_id":0,"CouponUser":"$couponUsed","Category":"$items.name","Prices":"$items.price"}},{"$limit":1}]):
    pprint(i)

{'Category': ['printer paper',
              'notepad',
              'pens',
              'backpack',
              'notepad',
              'envelopes',
              'envelopes',
              'binder'],
 'CouponUser': True,
 'Prices': [{'$numberDecimal': '40.01'},
            {'$numberDecimal': '35.29'},
            {'$numberDecimal': '56.12'},
            {'$numberDecimal': '77.71'},
            {'$numberDecimal': '18.47'},
            {'$numberDecimal': '19.95'},
            {'$numberDecimal': '8.08'},
            {'$numberDecimal': '14.16'}]}


In [55]:
for i in db.limited.find(): #limited는 제한컬렉션이다.
    print(i)

{'_id': ObjectId('61165e5dc6738d2fa99f5617'), 'age': '53'}
{'_id': ObjectId('61165e5dc6738d2fa99f5618'), 'age': '53'}
{'_id': ObjectId('61165e5dc6738d2fa99f5619'), 'age': '53'}


In [58]:
db.limited.insert_one({"age":"55"})

In [64]:
#인트로 변환하여 보여주기!
for i in db.limited.aggregate([{"$project":{"나이":{"$toInt":"$age"}}}]):
    print(i)

{'_id': ObjectId('61165e5dc6738d2fa99f5619'), '나이': 53}
{'_id': ObjectId('61165e6dc6738d2fa99f561a'), '나이': 54}
{'_id': ObjectId('61165e7ac6738d2fa99f561b'), '나이': 55}
